In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import random
from streaming.base import MDSWriter

os.chdir('../')

In [3]:
name = 'CARE_resampled50cluster_medium_withTax'

#ensure the directory exists
os.makedirs('data/{}'.format(name), exist_ok=True)

for split in ['train', 'val70', 'val90', 'test', 'train_common', 'train_rare', 'train_common_low_bacteria', 'test_common_low_bacteria']:
    split_df = pd.read_csv('data/{}/{}.csv'.format(name, split))
                    
    sequences = split_df['Sequence'].values.tolist()
    ECs = split_df['EC number'].values.tolist()
    taxes = split_df['Tax number'].values.tolist()

    output_dir = 'data/sharded_datasets/{}/{}'.format(name, split)
    #ensure the directory exists
    os.makedirs(output_dir, exist_ok=True)

    columns = {'sequence': 'str', 'ec': 'str', 'tax': 'str'} 
    with MDSWriter(out=output_dir, columns=columns) as out:
        for seq, ec, tax in zip(sequences, ECs, taxes):
            out.write({'sequence': seq, 'ec': ec, 'tax': tax})

### Save uniref sharded dataset (slow to load and save)

In [ ]:
df = pd.read_csv("data/raw_data/uniprot_enzyme_240617.tsv", sep='\t')
df = df.dropna(subset=['Sequence', 'EC number'])
df = df[~df['EC number'].str.contains(';')]
#shuffle the dataframe
df = df.sample(frac=1, random_state=42, replace=False)
sequences = df['Sequence'].values.tolist()
ECs = df['EC number'].values.tolist()

In [ ]:
output_dir = 'data/sharded_datasets/all_uniref/train'
#ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

columns = {'sequence': 'str', 'ec': 'str'}
with MDSWriter(out=output_dir, columns=columns) as out:
    for seq, ec in zip(sequences, ECs):
        out.write({'sequence': seq, 'ec': ec})